# Build database

In [1]:
import json
import re
import sqlite3
from collections import defaultdict
from pathlib import Path
from types import SimpleNamespace

import pandas as pd
from tqdm.notebook import tqdm

In [2]:
INAT_DIR = Path("..") / "data" / "inat"

In [3]:
args = SimpleNamespace(
    obs_dir=INAT_DIR / "obs",
    image_dir=INAT_DIR / "images",
    db=INAT_DIR / "inat.sqlite",
    taxa=INAT_DIR / "taxa.csv.gz",  # Loction of iNat taxon info
    plantae=47126,  # The taxon ID for the plant kingdom
)

## Insert taxa data

In [4]:
taxa_df = pd.read_csv(
    args.taxa,
    sep="\t",
    dtype={
        "taxon_id": int,
        "ancestry": str,
        "rank_level": float,
        "rank": str,
        "name": str,
        "active": bool,
    },
).fillna("")
taxa_df.shape

(1394173, 6)

In [5]:
taxa_df = taxa_df.loc[taxa_df["ancestry"].str.contains(f"/{args.plantae}/")]
taxa_df.shape

(364137, 6)

In [6]:
with sqlite3.connect(args.db) as cxn:
    taxa_df.to_sql("taxa", cxn, index=False, if_exists="replace")

## Functions for parsing iNaturalist observation JSON records

### Get record level data from the JSON record

In [7]:
def get_ids(rec, path):
    order = path.stem.replace("order_", "")
    return {
        "obs_id": rec["id"],
        "split": "",
        "order_": order,
    }

### Get phenology data from the JSON record

In [8]:
phenology = {
    13: "Flowering",
    14: "Fruiting",
    # 15: "Flower Budding",
    21: "No Evidence of Flowering",
}


def get_annotations(rec):
    annos = [a["controlled_value_id"] for a in rec["annotations"]]
    pheno = [phenology.get(a["controlled_value_id"]) for a in rec["annotations"]]
    pheno = ", ".join([a for a in pheno if a])
    return {
        "annotations": ",".join([str(a) for a in annos]),
        "phenology": pheno,
        "flowering": 1 if 13 in annos else 0,
        "fruiting": 1 if 14 in annos else 0,
        "neither": 1 if 21 in annos else 0,
    }

### Get taxon data from the JSON record

In [9]:
def get_taxon(rec):
    return {
        "taxon_id": rec["taxon"]["id"],
        "taxon": rec["taxon"]["name"],
        "ancestry": rec["taxon"]["ancestry"],
    }

### Get image data from the JSON record

In [10]:
def get_images(rec, obs_id):
    photos = []
    for photo in rec["photos"]:
        url = photo["url"].replace("square.", size)
        match = re.search(r"/(\d+)/[a-z]+\.([a-z]+)$", url, flags=re.I)

        if not match:
            continue

        image_path = args.image_dir / f"{match[1]}_{size}{match[2]}"

        if not image_path.exists():
            continue

        photos.append(
            {
                "photo_id": photo["id"],
                "obs_id": obs_id,
                "license": photo["license_code"],
                "path": str(image_path),
                "url": url,
            }
        )

    return photos

### Build database records for the observations and images.

In [11]:
size = "medium."  # medium large original

json_paths = sorted(args.obs_dir.glob("*.json"))

annotations = defaultdict(int)

observations = []
images = []

for json_path in tqdm(json_paths):
    with open(json_path) as in_file:
        data = json.load(in_file)

    for rec in data:
        obs_rec = get_ids(rec, json_path)
        obs_rec |= get_taxon(rec)
        obs_rec |= get_annotations(rec)

        if not obs_rec["phenology"]:
            continue

        imgs = get_images(rec, obs_rec["obs_id"])
        if not imgs:
            continue

        images += imgs
        observations.append(obs_rec)

  0%|          | 0/64 [00:00<?, ?it/s]

### Build the data frames and write them to the database.

In [12]:
obs_df = pd.DataFrame(observations)

img_df = pd.DataFrame(images)

In [13]:
with sqlite3.connect(args.db) as cxn:
    obs_df.to_sql("obs", cxn, index=False, if_exists="replace")
    img_df.to_sql("images", cxn, index=False, if_exists="replace")

## Count records for each order and each phenology category

In [19]:
counts = defaultdict(
    lambda: {
        "both": 0,
        "flowering": 0,
        "fruiting": 0,
        "neither": 0,
        "total": 0,
    }
)
grand = 0

In [20]:
for obs in observations:
    count = counts[obs["order_"]]
    count["total"] += 1
    grand += 1

    if obs["flowering"] and obs["fruiting"]:
        count["both"] += 1

    for key in ["flowering", "fruiting", "neither"]:
        if obs[key]:
            count[key] += 1

In [21]:
keys = sorted(counts.keys())
for taxon in keys:
    count = counts[taxon]
    print(
        f"{taxon:20} flowering ={count['flowering']: 5}, "
        f"fruiting ={count['fruiting']: 5}, neither ={count['neither']: 5}, "
        f"both = {count['both']: 5}, total = {count['total']: 5}"
    )
grand

Acorales             flowering =   28, fruiting =   29, neither =    9, both =     1, total =    65
Alismatales          flowering =  531, fruiting =  426, neither =  559, both =    55, total =  1461
Amborellales         flowering =    0, fruiting =    6, neither =    0, both =     0, total =     6
Apiales              flowering = 1020, fruiting =  331, neither =  212, both =   132, total =  1431
Aquifoliales         flowering =  210, fruiting =  924, neither =  322, both =     7, total =  1449
Arecales             flowering =  221, fruiting =  560, neither =  707, both =    42, total =  1446
Asparagales          flowering =  838, fruiting =  156, neither =  476, both =    28, total =  1442
Asterales            flowering = 1190, fruiting =  184, neither =  158, both =    74, total =  1458
Austrobaileyales     flowering =   96, fruiting =   39, neither =    7, both =     4, total =   138
Berberidopsidales    flowering =    9, fruiting =    0, neither =    6, both =     0, total =    15


64839